# Table of Contents
 <p><div class="lev1"><a href="#Task-1.-Compiling-Ebola-Data"><span class="toc-item-num">Task 1.&nbsp;&nbsp;</span>Compiling Ebola Data</a></div>
 <div class="lev1"><a href="#Task-2.-RNA-Sequences"><span class="toc-item-num">Task 2.&nbsp;&nbsp;</span>RNA Sequences</a></div>
 <div class="lev1"><a href="#Task-3.-Class-War-in-Titanic"><span class="toc-item-num">Task 3.&nbsp;&nbsp;</span>Class War in Titanic</a></div></p>

In [ ]:
from IPython.core.display import HTML
import pandas as pd
import numpy as np
import glob
import re
import string
import math

DATA_FOLDER = 'Data'  # Use the data folder provided in Tutorial 02 - Intro to Pandas.

## Task 1. Compiling Ebola Data

The `DATA_FOLDER/ebola` folder contains summarized reports of Ebola cases from three countries (Guinea, Liberia and Sierra Leone) during the recent outbreak of the disease in West Africa. For each country, there are daily reports that contain various information about the outbreak in several cities in each country.

Use pandas to import these data files into a single `Dataframe`.
Using this `DataFrame`, calculate for *each country*, the *daily average per month* of *new cases* and *deaths*.
Make sure you handle all the different expressions for *new cases* and *deaths* that are used in the reports.

This next cell is use to suppress Pandas FutureWarnings which always come up on ambiguity from Column/Index names

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

Helper functions to Format result tables from Dataframe.

We'll explain later why we use them. But we use cumulated data (which are increasing along the month). So what we do in this function is to divide the earliest and the latest data of each month by the number of elapsing days. From all the data point there is on each file.

To do so we have to update index to a sortable value representing only month and year (integer)


In [ ]:
TOTAL_DEATH_DAILY_AVG = 'Total Death (daily avg.)'
NEW_CASES_DAILY_AVG = 'New cases (daily avg.)'

# We create a numerical index to sort by year month.
def only_year_month(x):
    return x.year * 100 + x.month

def avg_table_daily(table):
    new_indexes = reversed(list(set(table.index.map(only_year_month))))
    avg_result = pd.DataFrame(index=new_indexes, columns=[TOTAL_DEATH_DAILY_AVG, NEW_CASES_DAILY_AVG])
    old_index = table.index
    i = 0
    while i < len(old_index):
        new_ind = only_year_month(old_index[i])
        num_days = old_index[i+1].day - old_index[i].day + 1
        
        [x,y] = table['Total death'][old_index[i]:old_index[i+1]]
        avg_result[TOTAL_DEATH_DAILY_AVG][new_ind] = (y-x) / num_days
        
        [x,y] = table['New cases (cumul.)'][old_index[i]:old_index[i+1]]
        avg_result[NEW_CASES_DAILY_AVG][new_ind] = (y-x) / num_days
        i =  i + 2

    avg_result = avg_result.sort_index()
        
    return avg_result

Helper function to check headers from a list of files. It display set of different headers to be able to visually check if columns are missings

In [ ]:
def check_headers(files):
    list_ = []
    for file_ in files:
        df = pd.read_csv(file_, header=None)
        list_.append(df.head(n=1))

    frame = pd.concat(list_)
    # Transpose to not drop record and better visualisation
    return frame.drop_duplicates().T

Helper function checking if dates are unique per file

In [ ]:
def are_date_unique_per_file(files, lowercase=False):
    cols=['Date']
    if lowercase:
        cols=['date']
    for file_ in files:
        df = pd.read_csv(file_, usecols=cols)
        #print(file_, df.shape)
        if df.drop_duplicates().shape[0] > 1:
            return False
    return True

## Liberia

### Check structures of the files

#### Headers

In [ ]:
allFiles = glob.glob(DATA_FOLDER + '/ebola/liberia_data/*.csv')

check_headers(allFiles)

In [ ]:
are_date_unique_per_file(allFiles)

We wanted to use first the "Newly reported deaths" column, but they were disrepedancies with the cumulated data, and the newly reported data did not indicate if it was confirmed, suspected, etc. For the "Total death/s in confirmed cases",  less than half of the files didn't contain the value for this field. So either we switch to no data or more data but overestimation. Then we decided to switch to cumulated "Total death/s in confirmed, probable, suspected cases".

We also normalize most of the column by replacing new lines and repeating space since this is what we spotted


In [ ]:
NEW_CASES = "New case/s (confirmed)"
TOTAL_NEW_CASES = "Total confirmed cases"
TOTAL_DEATH = "Total death/s in confirmed, probable, suspected cases"
NEWLY_REPORTED_DEATH = "Newly reported deaths"


# most of error are extra \n and spaces
def check_column_in_files(files, column_name, lower_case=False, descript=False):
    # Added lower_case boolean not to rewrite function as Sierre Leone data are lowercased.
    # Added descript boolean not to rewrite function as Guinea data are different.
    cols=['Variable', 'Date']
    if lower_case:
        cols=['variable', 'date']
    if descript:
        cols=['Description', 'Date']
    
    for file_ in allFiles:
        df = pd.read_csv(file_, index_col=cols).fillna(value=0)
        
        # normalize column names
        new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
        df.index = df.index.set_levels(levels=new_values, level=0)
    
        is_column_present = column_name in df.index
        
        if not is_column_present:
            return False
        
    return True


to_check = [NEW_CASES, TOTAL_DEATH, NEWLY_REPORTED_DEATH, TOTAL_NEW_CASES]
list(map(lambda x: check_column_in_files(allFiles, x), to_check))

So, the both lines are available in each file, however a quick check in the files indicates that the cell may be empty.

Below, we found out that there is many NA values for the lasts months, only totals are available which mixes confirmed, probable and suspected cases. We have chosen along the way to consider only *confirmed cases*, so we are going to drop those data. First, let's check how many data we are going to drop.

Also, there is discrependencies between New deaths reported and cumulated data.

For the new cases, we also are going to use the cumulated data, except for the last file, were it seems that the cumulated data has moved to the new cases column. We patch manually this.

## More about cumulated data

We pick the earliest and the latest day in each month and we do the difference between the two to compute the average death and new cases per month. What happen is that the latest and the earliest day in the month do not match respectively with the end and the beginning of the month. We have two options from there. Assuming with have data from the 4th September to the 25th. And next month the data are starting at the 3rd of October. Either we don't take in account the gap or we try to interpolate the data. But we may not know the distribution of the new cases/death between each day so we found it was dangerous to interpolate. We will only do the averages on the data we have.

In the `2014-10-04-v142.csv` file, there is two Total deaths line, we keep only the second one (because it is the largest)

For the cumulated data, they are not always increasing, and we apply a `rolling` operation to correct values that decrease by replacing the ovbviously wrong value (nobody resuscitates!) by its precendent value. It may be incorrect but still better than discarding a value. We checked that it doesn't happen at the end or the beginning of a month. Rolling discard the very first and very last value, so we manually replace them.

In [ ]:
allFiles = glob.glob(DATA_FOLDER + '/ebola/liberia_data/*.csv')

list_ = []

for file_ in allFiles:
    df = pd.read_csv(file_, index_col=['Variable', 'Date'])
    
    # Normalize variable column
    new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
    df.index = df.index.set_levels(levels=new_values, level=0)
    
    data = df[["National"]]

    total_deaths = data.loc[TOTAL_DEATH]
    new_cases = data.loc[NEW_CASES]
    total_cases = data.loc[TOTAL_NEW_CASES]
    
    concatenated = pd.concat([total_deaths.tail(n=1), new_cases, total_cases], axis=1)
    concatenated.columns = ["Total death", "New cases", "New cases (cumul.)"]
    list_.append(concatenated)
    
frame = pd.concat(list_)

# Manual patch
frame["New cases (cumul.)"]["12/9/2014"] = frame["New cases"]["12/9/2014"]
frame.index = pd.to_datetime(frame.index)

# To remove decreasing number in an increasing serie, we compare two number in
# a moving window fashion and replace a number with its previous row content
# if it is smaller (since cumulated value should only increase)
def remove_outliers(x,y):
    if(x > y):
        return x
    return y

frame["corr"] = frame["New cases (cumul.)"].rolling(window=2).apply(lambda x: remove_outliers(x[0], x[1]))

# We keep only the first 
frame["Month"] = frame.index.map(lambda x: x.month).astype(int)
frame["Day"] = frame.index.map(lambda x: x.day).astype(int)
frame.index = [frame["Month"], frame["Day"]]

a = pd.concat(
    [frame.groupby(['Month'])['Day'].min().reset_index(),
    frame.groupby(['Month'])['Day'].max().reset_index()]
).sort_index()
result = pd.merge(a, frame, on=["Month", "Day"], how="left")

result.index = pd.to_datetime(result[['Month','Day']].apply(lambda x : '{}-{}-2014'.format(x[0],x[1]), axis=1))
result = result.drop(["Month", "Day"], axis=1)

result.loc["2014-06-16"]["corr"] = result.loc["2014-06-16"]["New cases (cumul.)"]
result.loc["2014-12-09"]["corr"] = result.loc["2014-12-09"]["New cases (cumul.)"]
result = result.drop(["New cases (cumul.)", "New cases"], axis=1)
result = result.rename(columns = {'corr':'New cases (cumul.)'})
result

As explained before we will only consider the cumulative results as they are much more coherent et complete.

#### Now putting all that togheter and averaging daily we get this results for the Liberia data:

In [ ]:
avg_result_L = avg_table_daily(result)
avg_result_L['Country'] = 'Liberia'
avg_result_L.set_index("Country", append=True)
avg_result_L

## Sierre Leone

### Check structures of the files

#### Headers

In [ ]:
allFiles = glob.glob(DATA_FOLDER + '/ebola/sl_data/*.csv')
check_headers(allFiles)

In [ ]:
are_date_unique_per_file(allFiles, True)

For the Sierra Leone we decided to keep the *confirmed* number for the new cases and death, as it seemed to make more sense to keep only the amount of infected and dead we were sure about.

In [ ]:
TOTAL_NEW_CASES = "cum_confirmed"
TOTAL_DEATH = "death_confirmed"

to_check = [TOTAL_DEATH, TOTAL_NEW_CASES]
list(map(lambda x: check_column_in_files(allFiles, x, True, False), to_check))

As the total deaths data was missing from the last file for december (13-12-2014) we decided to keep the amounts (for every column) shown in the last file that was complete (05-12-2014). This is why we do not take the last two files in the calculations.

In [ ]:
list_ = []

for file_ in allFiles[:len(allFiles) - 2]:
    df = pd.read_csv(file_, index_col=['variable', 'date'], thousands=',')
    
    # Normalize variable column
    new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
    df.index = df.index.set_levels(levels=new_values, level=0)
    
    data = df[["National"]]
    
    # Remove commas in the series
    total_deaths = data.loc[TOTAL_DEATH].replace(regex=True,to_replace=r',',value=r'').fillna(value=0).astype(int)
    total_cases = data.loc[TOTAL_NEW_CASES].replace(regex=True,to_replace=r',',value=r'').fillna(value=0).astype(int)
        
    concatenated = pd.concat([total_deaths.tail(n=1), total_cases], axis=1)
    concatenated.columns = ["Total death", "New cases (cumul.)"]
    
    list_.append(concatenated)
    
    
    
frame = pd.concat(list_)
frame.index = pd.to_datetime(frame.index)
frame["Month"] = frame.index.map(lambda x: x.month).astype(int)
frame["Day"] = frame.index.map(lambda x: x.day).astype(int)
frame.index = [frame["Month"], frame["Day"]]

a = pd.concat(
    [frame.groupby(['Month'])['Day'].min().reset_index(),
    frame.groupby(['Month'])['Day'].max().reset_index()]
).sort_index()
result = pd.merge(a, frame, on=["Month", "Day"], how="left")

result.index = pd.to_datetime(result[['Month','Day']].apply(lambda x : '{}-{}-2014'.format(x[0],x[1]), axis=1))
result = result.drop(["Month", "Day"], axis=1)

result

#### Now putting all that togheter and averaging daily we get this results for the Sierra Leone data:

In [ ]:
avg_result_SL = avg_table_daily(result)
avg_result_SL['Country'] = 'Sierra Leone'
avg_result_SL

## Guinea

### Check structures of the files

#### Headers

In [ ]:
allFiles = glob.glob(DATA_FOLDER + '/ebola/guinea_data/*.csv')
check_headers(allFiles)

In [ ]:
are_date_unique_per_file(allFiles)

We can see that three columns are always available: Date, Description and Totals. We decided to use the total cumulative confirmed amount for deaths and new cases.

In [ ]:
TOTAL_NEW_CASES = "Total cases of confirmed"
TOTAL_DEATH = "Total deaths of confirmed"

to_check = [TOTAL_DEATH, TOTAL_NEW_CASES]
list(map(lambda x: check_column_in_files(allFiles, x, False, True), to_check))

In [ ]:
list_ = []

for file_ in allFiles[:len(allFiles)-1]:
    df = pd.read_csv(file_, index_col=['Description', 'Date'])
    
    # Normalize variable column
    new_values = list(map(lambda x: re.sub("\s\s+" , " ", x.replace("\n", "")), df.index.levels[0].values.tolist()))
    df.index = df.index.set_levels(levels=new_values, level=0)
    
    data = df[["Totals"]]
    
    total_deaths = data.loc[TOTAL_DEATH].fillna(value=0).astype(int)
    total_cases = data.loc[TOTAL_NEW_CASES].fillna(value=0).astype(int)
    
    concatenated = pd.concat([total_deaths.tail(n=1), total_cases], axis=1)
    concatenated.columns = ["Total death", "New cases (cumul.)"]
    
    list_.append(concatenated)
    
    
    
frame = pd.concat(list_)
frame.index = pd.to_datetime(frame.index)
frame["Month"] = frame.index.map(lambda x: x.month).astype(int)
frame["Day"] = frame.index.map(lambda x: x.day).astype(int)
frame.index = [frame["Month"], frame["Day"]]

a = pd.concat(
    [frame.groupby(['Month'])['Day'].min().reset_index(),
    frame.groupby(['Month'])['Day'].max().reset_index()]
).sort_index()
result = pd.merge(a, frame, on=["Month", "Day"], how="left")

result.index = pd.to_datetime(result[['Month','Day']].apply(lambda x : '{}-{}-2014'.format(x[0],x[1]), axis=1))
result = result.drop(["Month", "Day"], axis=1)

result

The guinea data for October 2014 only contained one day of data. We decided to drop it as it doesn't make much sense to return a daily average per month on one day.

#### Now putting all that togheter and averaging daily we get this results for the Guinea data:

In [ ]:
avg_result_G = avg_table_daily(result)
avg_result_G['Country'] = 'Guinea'
avg_result_G

### Putting everything together

In [ ]:
def yearmonth_int_to_string(x):
    return "{}-{}".format(x // 100, x % 100)

In [ ]:
total_result = pd.concat([avg_result_L, avg_result_SL, avg_result_G])
total_result = total_result.set_index([total_result['Country'], total_result.index]).drop('Country', axis=1)
total_result.index.set_levels(total_result.index.levels[1].map(yearmonth_int_to_string), 1, inplace=True)
total_result

## Task 2. RNA Sequences

In the `DATA_FOLDER/microbiome` subdirectory, there are 9 spreadsheets of microbiome data that was acquired from high-throughput RNA sequencing procedures, along with a 10<sup>th</sup> file that describes the content of each. 

Use pandas to import the first 9 spreadsheets into a single `DataFrame`.
Then, add the metadata information from the 10<sup>th</sup> spreadsheet as columns in the combined `DataFrame`.
Make sure that the final `DataFrame` has a unique index and all the `NaN` values have been replaced by the tag `unknown`.

## Solution

To solve this exercice we will first load the metadata and modify the NA value to the *unknown* tag. After that we will load every MIDx.xls file in a separate DataFrame and store them in a list to be able to concat them together. While we load the files we will add a new column call *BARCODE* and store which file every row comes from. This will allow us to use a simple merge to fill the metadatas in for each line. Finaly to keep unique indexes we will use the tupple [*BARCODE, TAXON*]

In [ ]:
DATA_FILE = DATA_FOLDER + '/microbiome/'
METADATA = 'metadata'
MID = 'MID'
XLS = '.xls'

In [ ]:
metadata = pd.read_excel(DATA_FILE + METADATA + XLS).fillna('unknown')
metadata

In [ ]:
temp_df_list = []
for x in range(9):
    temp_df = pd.read_excel(DATA_FILE + MID + str(x+1) + XLS, header=None)
    temp_df['BARCODE'] = pd.Series([MID + str(x+1)]*len(temp_df))
    temp_df_list.append(temp_df)
taxons = pd.concat(temp_df_list)
taxons.columns = ['TAXON', 'COUNT', 'BARCODE']
taxons.head()

In [ ]:
final = taxons.merge(metadata, on='BARCODE', how='left').set_index(['BARCODE', 'TAXON'])
final.head()

In [ ]:
final.tail()

We can check that the index is unique and that there is no more NaN values


In [ ]:
final.index.is_unique

In [ ]:
is_null = final.isnull()
is_null[(is_null["COUNT"]) | (is_null["GROUP"]) | (is_null["SAMPLE"])].shape[0]

## Task 3. Class War in Titanic

Use pandas to import the data file `Data/titanic.xls`. It contains data on all the passengers that travelled on the Titanic.

In [ ]:
HTML(filename=DATA_FOLDER+'/titanic.html')

For each of the following questions state clearly your assumptions and discuss your findings:
1. Describe the *type* and the *value range* of each attribute. Indicate and transform the attributes that can be `Categorical`. 
2. Plot histograms for the *travel class*, *embarkation port*, *sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 
3. Calculate the proportion of passengers by *cabin floor*. Present your results in a *pie chart*.
4. For each *travel class*, calculate the proportion of the passengers that survived. Present your results in *pie charts*.
5. Calculate the proportion of the passengers that survived by *travel class* and *sex*. Present your results in *a single histogram*.
6. Create 2 equally populated *age categories* and calculate survival proportions by *age category*, *travel class* and *sex*. Present your results in a `DataFrame` with unique index.

In [ ]:
#Load and cleanup data
frame = pd.read_excel(DATA_FOLDER+'/titanic.xls')
frame.ticket = frame.ticket.map(lambda tickets: str(tickets).strip('{}{}'.format(string.ascii_letters, string.punctuation)).split(' ')[-1])
frame.ticket = frame.ticket.map(lambda tickets: 0 if len(tickets)==0 else int(tickets))
frame

In [ ]:
# 1. Describe the *type* and the *value range* of each attribute.

print("\npclass \n range: {}\n type: {}".format(frame.pclass.unique(), frame.pclass.dtype))
print("\nsurvived \n range: {}\n type: {}".format(frame.survived.unique(), frame.survived.dtype))
print("\nsex \n range: {}\n type: {}".format(frame.sex.unique(), frame.sex.dtype))
print("\nembarked \n range: {}\n type: {}".format(frame.embarked.unique(), frame.embarked.dtype))
print("\nparch \n range: {}-{}\n type: {}".format(frame.parch.min(), frame.parch.max(), frame.parch.dtype))
print("\nsibsp \n range: {}-{}\n type: {}".format(frame.sibsp.min(), frame.sibsp.max(), frame.sibsp.dtype))
print("\nage \n range: {}-{}\n type: {}".format(frame.age.min(), frame.age.max(), frame.age.dtype))
print("\nticket \n range: {}-{}\n type: {}".format(frame.ticket.min(), frame.ticket.max(), frame.ticket.dtype))
print("\nfare \n range: {}-{}\n type: {}".format(frame.fare.min(), frame.fare.max(), frame.fare.dtype))
print("\nbody \n range: {}-{}\n type: {}".format(frame.body.min(), frame.body.max(), frame.body.dtype))
print("\nboat \n range: {}\n type: {}".format(frame.boat.unique(), frame.boat.dtype))
print("\ncabin \n range: {}\n type: {}".format(frame.cabin.unique(), frame.cabin.dtype))
print("\nname \n range: {}\n type: {}".format(frame.name.unique(), frame.name.dtype))
print("\nhome.dest \n range: {}\n type: {}".format(frame['home.dest'].unique(), frame['home.dest'].dtype))

<b>pclass</b>, <b>survived</b> and <b>sex</b> can be categorical values. These are deviding the dataset to reasonably smaller subsets
<b>embarked</b> can be also considered as categorical value if we decide to eliminate the samples that have 'nan' value in this column

In [ ]:
#Transform the attributes that can be `Categorical`. 
frame['pclass'] = frame.pclass.astype('category')
frame['survived'] = frame.survived.astype('category')
frame['sex'] = frame.sex.astype('category')
frame['embarked'] = frame.embarked.astype('category')

In [ ]:
#Drop columns that won't be used furhter
clean_frame = frame.drop(['sibsp', 'parch', 'ticket', 'fare', 'body', 'home.dest', 'boat', 'name'], axis=1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.style.use('ggplot')

#Plot histograms for the *travel class*, *embarkation port*,
#*sex* and *age* attributes. For the latter one, use *discrete decade intervals*. 

clean_frame['sex'].value_counts().plot(kind='bar', color=['r','g'], title='Grouped by sex')
plt.show()

clean_frame['pclass'].value_counts().plot(kind='bar', color=['r','g','b'], title='Grouped by travel class')
plt.show()

clean_frame['embarked'].value_counts().plot(kind='bar', color=['r','g','b'], title='Grouped by embarkation port')
plt.show()

pd.cut(clean_frame.age, [0,10,20,30,40,50,60,70,80]).value_counts().plot(kind='bar', title='Grouped by decade')
plt.show()

In [ ]:
for c in clean_frame.cabin.values:
    if len(str(c).split(' '))>1:
        print(c) 

According to [this](https://www.encyclopedia-titanica.org/titanic-deckplans/g-deck.html) map, we assume that the starting letter for each value represent the floor on which the person stays. For the cases where we have for exemple 'F G63' we assume that the person stayed on floor F in part G cabin 63. When there are more than one value as cabin for the same person we noticed that these cabins are always on the same floor (showed above). 

In [ ]:
#Calculate the proportion of passengers by cabin floor. Present your results in a pie chart

#Add new column named floor
clean_frame['floor'] = clean_frame.cabin.dropna().map(lambda f: f[0])
#Make it categorical 
clean_frame['floor'] = clean_frame.floor.astype('category')
#Show the piechart
clean_frame['floor'].value_counts().plot(kind='pie', title='Grouped by cabin floor')
plt.show()

In [ ]:
#For each travel class, calculate the proportion of the passengers that survived.
#Present your results in pie charts.
clean_frame[clean_frame.survived==1]['pclass'].value_counts().plot(kind='pie', title='Survived proportions from each class')
plt.show()
clean_frame.groupby(['pclass', 'survived']).pclass.value_counts().plot(kind='pie', title='Survived(1) vs. Dead(0) all together')
plt.show()
clean_frame[clean_frame.pclass==1]['survived'].value_counts().plot(kind='pie', title='Survived(1) from class 1')
plt.show()
clean_frame[clean_frame.pclass==2]['survived'].value_counts().plot(kind='pie', title='Survived(1) from class 2')
plt.show()
clean_frame[clean_frame.pclass==3]['survived'].value_counts().plot(kind='pie', title='Survived(1) from class 3')
plt.show()

In [ ]:
#Calculate the proportion of the passengers that survived by travel class and sex.
#Present your results in a single histogram.
grouped = clean_frame[clean_frame.survived==1].groupby(['pclass', 'sex'])

grouped.survived.value_counts().plot(kind='bar', color=['r','g','b','m','y', 'c'], title='Survived passengers by sex and travel class')
plt.show()

In [ ]:
#Create 2 equally populated age categories and calculate survival proportions by age category, travel class and sex. 
by_ages = pd.qcut(clean_frame.age, 2)
clean_frame['by_ages'] = by_ages

grouped = clean_frame[clean_frame.survived==1].groupby(['pclass', 'sex', 'by_ages'])
grouped.survived.value_counts().plot(kind='bar', color=['r','g','b','m','y', 'c'], title='Survived passengers by sex, travel class and age category')
plt.show()

In [ ]:
#Present your results in a DataFrame with unique index.
clean_frame['id'] = range(len(clean_frame))
clean_frame = clean_frame.set_index(['id'])
clean_frame